<a href="https://colab.research.google.com/github/dbaglodi/AI-For-AccessCORPS/blob/main/table_extraction_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#!pip install Pillow pytesseract google-generativeai
!pip install Pillow google-generativeai

In [ ]:
# @title
import os
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

# Image to Table (HTML)
Run this cell to upload an image of a table.

In [ ]:
# @title
import os
from PIL import Image
#import pytesseract
import io
from google.colab import files, output, userdata
import google.generativeai as genai

# Set your Gemini API key
#os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

#def install_tesseract_and_ocr():
#    """Installs Tesseract OCR and its Python wrapper in Colab."""
#    try:
#        !apt-get update  # Update package list
#        !apt-get install -y tesseract-ocr
#        !pip install pytesseract
#        print("Tesseract OCR installed successfully.")
#        return True
#    except Exception as e:
#        print(f"Error installing Tesseract OCR: {e}")
#        return False

def extract_table_from_image(image_bytes):
    """Extracts a table from image bytes using OCR and Gemini API (using google.generativeai)."""
    try:
        img = Image.open(io.BytesIO(image_bytes))
        #extracted_text = pytesseract.image_to_string(img)

        prompt = f"""
        Please analyze the following image of a table and convert it to a WCAG compliant table in html format. Make sure to appropriately modify rowspan and/or colspan to account for split cells. Do not add anything extra.
        """
        genai.configure(api_key=os.environ["GEMINI_API_KEY"])

        model = genai.GenerativeModel('gemini-2.0-flash')

        response = model.generate_content([prompt, img])

        gemini_output = response.text  # Access the result directly

        return f"{gemini_output[gemini_output.index('<table>'):gemini_output.rindex('</table>')+8]}\n"
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def process_uploaded_image():
    """Handles image upload and table extraction in Colab."""
    #if not install_tesseract_and_ocr():  # Install Tesseract first
    #    return  # Exit if installation fails

    try:
        uploaded = files.upload()

        if uploaded:
            full_data = '<html>\n'
            for filename, data in uploaded.items():
                image_bytes = data
                table_data = extract_table_from_image(image_bytes)

                if table_data:
                    output.clear()
                    #table_data = f"<html>\n{table_data[table_data.index('<'):table_data.rindex('>')+1]}\n<\html>"
                    #print(table_data)
                    full_data += table_data
                else:
                    print("Table extraction failed.")
            full_data += '</html>'
            print(full_data)
            return full_data
        else:
          print("No file uploaded")

    except Exception as e:
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    table_data = process_uploaded_image()  # Call the processing function directly

https://mconverter.eu/convert/html/docx/ <br>
https://convertio.co/html-pptx/

# MConverter Conversion to docx
Run this section to convert the images of tables to a Word document with tables.

In [ ]:
with open("table_data.html", "w") as file:
    file.write(table_data)

In [ ]:
response = !curl https://api.mconverter.eu/v1/start_conversion.php -X POST -F "api_key=usXGCWEO9GWncAU8LrAOQf8nzXzTeddr" -F "target_format=docx" -F "source=@/content/table_data.html"
response

In [ ]:
id = response[0]
id = int(id[id.index('id')+4:id.index(',')])
id

In [ ]:
while True:
  status = !curl https://api.mconverter.eu/v1/check_progress.php -X POST -F "api_key=usXGCWEO9GWncAU8LrAOQf8nzXzTeddr" -F "conversion_id={id}"
  status = status[0]
  start = status.index('status') + 9
  status = status[start:start+8]
  print(status)
  if status == "finished":
    break

In [ ]:
!curl https://api.mconverter.eu/v1/get_file.php -o table_data.docx -J -X POST -F "api_key=usXGCWEO9GWncAU8LrAOQf8nzXzTeddr" -F "conversion_id={id}"

In [ ]:
from google.colab import files
files.download('table_data.docx')

Source for dataset:
https://www.kaggle.com/datasets/rhtsingh/general-table-recognition-dataset?resource=download